In [1]:
%load_ext autoreload
%autoreload 2
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lark import Lark
import time 

# prepare dataset

In [2]:
def sklearn_to_df(sklearn_dataset):
    """ 
    Convert sklearn dataset to a dataframe, the class-label is renamed to "target"
    """
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df


target = "target"
dataset = datasets.load_iris()
dataset[target] = np.where(dataset[target]==2, 0, dataset[target])



# get df
data = sklearn_to_df(dataset)

# get X,y
X = data.drop(['target'], axis=1)
y = data['target']

X.columns = ['sepal length', 'sepal width', 'petal length', 'petal width']


# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # 70% training and 30% test




# train a black-box

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

clf_rf=RandomForestClassifier(n_estimators=100)
clf_mlp = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf_rf.fit(X_train,y_train)
clf_mlp.fit(X_train,y_train)

y_pred=clf_mlp.predict(X_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


print(clf_rf.predict([[3, 5, 4, 2]]))
print(clf_mlp.predict([[3, 5, 4, 2]]))

Accuracy: 0.9333333333333333
[1]
[0]


# create a random generator

In [4]:

import random
def random_generator(X):
    num_attributes = len(X.columns)
    x=[]
    for i in range(num_attributes):
        x.append(random.uniform(X[X.columns[i]].min(),X[X.columns[i]].max()))
    return x

for i in range(5):
    x = random_generator(X)
    print(x)
    print(clf_rf.predict([x]))
    print(clf_mlp.predict([x]))

[7.8168659283006185, 2.146322080077088, 5.038241224325036, 0.36593311647039706]
[0]
[1]
[7.586024143944473, 2.816257156027497, 6.443621555661401, 0.29878300446239736]
[0]
[1]
[5.659587313486211, 3.090570560561754, 3.047413696747793, 0.7190902567353016]
[1]
[0]
[5.039674180189529, 2.392833340404275, 2.5475575390850462, 2.2801594003677885]
[0]
[0]
[5.758233840103882, 3.0313050737047007, 4.958412591743274, 1.7936062728180715]
[0]
[0]


# implementation of learner

In [5]:
from sygus_if import SyGuS_IF
# X = [
#     [1,2,1,1],
#     [-1.1,1,1,42]
# ]
# y = [
#     1,
#     0
# ]

In [6]:
# start_ = time.time()



# y_pred_test = sgf.predict_z3(X_test)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
# y_pred_train = sgf.predict_z3(X_train)
# print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
# print(y_pred_test)
# print(time.time() - start_)

In [7]:


# start_ = time.time()

# y_pred_test = sgf.predict(X_test, y_test)
# print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
# y_pred_train = sgf.predict(X_train, y_train)
# print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))

# print(time.time() - start_)

# implementation of the query

In [8]:
import query
def predict_function_query(x):
    return True

q = query.Query(model = None, prediction_function = predict_function_query)
q.classify_example([3, 5, 4, 2])

True

# implementation of black-box

In [9]:
from blackbox import BlackBox
bb = BlackBox(clf_mlp, clf_mlp.predict)
bb.classify_example([3, 5, 4, 2])

False

# implementation of learner

(define-fun func ((x_0 Real) (x_1 Real) (x_2 Real) (x_3 Real)) Real 1)


# implementation of teacher

In [33]:
from teacher import Teacher
from learner import Learner
sgf = SyGuS_IF()
l = Learner(model = sgf, prediction_function = sgf.predict_z3, train_function = sgf.fit, X = X_train.iloc[[0]].values.tolist() , y = y_train.iloc[[0]].values.tolist() )
l.fit()


t = Teacher(max_iterations=50)
new_l, flag = t.teach(blackbox = bb, learner = l, query = q, random_example_generator = random_generator, params_generator = X)
print(flag)
print(new_l.model.synthesized_function)
print(new_l.y)

100%|██████████| 50/50 [00:13<00:00,  3.80it/s]False
(define-fun func ((x_0 Real) (x_1 Real) (x_2 Real) (x_3 Real)) Real (ite (>= x_3 (/ 1 5)) (ite (>= x_0 (/ 13 3)) (ite (>= x_3 (/ 1 4)) (ite (>= x_3 (/ 1 3)) (ite (>= x_0 (/ 9 2)) (ite (< x_2 6) (ite (< x_3 2) (ite (< x_3 (/ 3 2)) (ite (< x_2 5) (ite (>= x_2 (/ 4 3)) (ite (>= x_1 (/ 5 2)) (ite (>= x_0 5) (ite (>= x_1 3) (ite (< x_1 4) (ite (< x_3 1) (ite (>= x_3 (/ 1 2)) (ite (>= x_2 3) 1 0) (ite (< x_0 6) 0 (ite (>= x_2 3) 1 0))) (ite (< x_2 4) 0 (ite (< x_0 6) 0 1))) (ite (< x_3 1) (ite (< x_3 (/ 1 2)) (ite (>= x_2 4) 1 0) 0) (ite (< x_2 4) 0 1))) (ite (< x_3 1) 1 (ite (< x_2 4) 0 1))) (ite (>= x_1 3) 0 (ite (< x_3 1) (ite (< x_3 (/ 1 2)) (ite (< x_2 2) 0 1) 0) 1))) (ite (>= x_0 5) (ite (< x_3 1) (ite (< x_3 (/ 1 2)) 1 (ite (>= x_2 4) 1 (ite (< x_0 6) 0 1))) 1) 0)) (ite (>= x_1 (/ 5 2)) 0 (ite (>= x_0 5) 1 0))) (ite (>= x_0 5) (ite (>= x_1 3) (ite (< x_1 4) (ite (< x_0 6) 0 (ite (>= x_2 3) 1 0)) (ite (< x_3 1) (ite (< x_3 (/ 1 2)) (

In [34]:
start_ = time.time()
y_pred_test = new_l.model.predict_z3(X_test.values.tolist())
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
y_pred_train = new_l.model.predict_z3(X_train.values.tolist())
print("Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print(time.time() - start_)

Accuracy: 0.6
Accuracy: 0.837037037037037
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
1.9271278381347656
